In [1]:
!conda activate asl

'conda' is not recognized as an internal or external command,
operable program or batch file.


# Exam DS807

## Preparation of data

### Labeling Data and converting images to array values

In [2]:
#import packages for labelling and converting imagery data
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import asarray
from sklearn import svm
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier

In [3]:
##############
# Func for creating 1D image arrays

def load_image_function(path):
    images = [] # empty list placeholder
    labels = [] # empty list placeholder
    container = [] # container to validate correct labels
    for filename in os.listdir(path):
        container.append(filename) # add filename to container     
        CCDY_img = load_img(path + f'/{filename}', target_size = (56, 106),
        color_mode="grayscale")                  
        CCDY_img = img_to_array(CCDY_img)
        images.append(CCDY_img) # append the photo to the images. The images list contains a list of arrays       
    return asarray(images), container; # not interested in list of arrays, but array containing lists. Asarray does this. Returns three arrays


In [4]:
################
# Func for creating df with classes and 3d img arrays

def load_img_data ():
    # Start: creatign classes ons string_digits
    # load string digits

    #os.chdir(path_string_digits)

    string_digits = pd.read_csv('DIDA_12000_String_Digit_Labels.csv', 
                 header = None, 
                 names=["index", "string"])
    # create empty class columns
    string_digits['CC'] = 0
    string_digits['D'] = 0
    string_digits['Y'] = 0
    string_digits = string_digits.astype(str)
    # Iterate string digits and append classes
    for i, row in string_digits.iterrows():
        if len(row['string']) != 4:
            row['CC'] = '1'
            row['D'] = '10'
            row['Y'] = '10'
        else:
            row['D'] = row['string'][2]
            row['Y'] = row['string'][3]
            if row['string'][0:2] == '18':
                row['CC']='0'
            else:
                row['CC']='1'
    # End of class labeling on string_digits.
    #
    # Start: create img_df containing scaled images as 1D tensors
    # Convert imagery to 1D arrays with tagged file names

    #os.chdir(path_images)
    image_array, filename = load_image_function('DIDA_12000_String_Digit_Images/DIDA_1')

    # and convert to a img_df
    img_df = pd.DataFrame({'filename': filename, 'img_value': list(image_array)}, 
                          columns=['filename', 'img_value'])
    # Create proper index value in img_df to allow merge on string_digits
    img_df['index'] = img_df['filename']
    for i, row in img_df.iterrows():
        row['index'] = str(img_df['index'][i]).split('.')[0]
    img_df
    # End of creating img_df
    #
    # Start: merge img_df with string_digits
    # match index type on dataframes to merge
    string_digits['index'] = string_digits['index'].astype(int)
    img_df['index'] = img_df['index'].astype(int)
    # Merge dataframes
    df_img_classes = string_digits.merge(img_df)
    # Rearrange order of dataframe
    df_img_classes = df_img_classes.reindex(columns= ['index', 'string', 'CC', 'D', 'Y', 'img_value', 'filename'])
    # End og merging data frames
    return df_img_classes


In [5]:
#load data and shuffle them around
df = load_img_data()
df = df.sample(frac=1).reset_index(drop=True)
df

,index,string,CC,D,Y,img_value,filename
0,9406,1835,0,3,5,"[[[171.0], [171.0], [171.0], [166.0], [177.0],...",9406.jpg
1,905,1836,0,3,6,"[[[133.0], [135.0], [136.0], [136.0], [135.0],...",905.jpg
2,4104,1822,0,2,2,"[[[163.0], [166.0], [167.0], [164.0], [160.0],...",4104.jpg
3,11719,1809,0,0,9,"[[[179.0], [177.0], [178.0], [178.0], [176.0],...",11719.jpg
4,8343,1819,0,1,9,"[[[157.0], [155.0], [158.0], [158.0], [160.0],...",8343.jpg
...,...,...,...,...,...,...,...
11995,4934,1820,0,2,0,"[[[142.0], [144.0], [145.0], [144.0], [146.0],...",4934.jpg
11996,2673,1803,0,0,3,"[[[129.0], [128.0], [127.0], [127.0], [126.0],...",2673.jpg
11997,8988,1823,0,2,3,"[[[168.0], [170.0], [169.0], [160.0], [156.0],...",8988.jpg
11998,8087,1815,0,1,5,"[[[105.0], [67.0], [72.0], [100.0], [123.0], [...",8087.jpg


### Defining fixed train, test and valdiation sets.

In [6]:
df = df.drop(columns=['index','string','filename'])

### Split train, val, test

#### Subsetting df

In [7]:
x_train = df.loc[0:7679]
x_val = df.loc[7680:9599]
x_test = df.loc[9600:11999]

#### Creating labels

In [8]:
#### training labels ###
Y_train = x_train['Y']
Y_train = Y_train.astype(np.uint8)
D_train = x_train['D']
D_train = D_train.astype(np.uint8)
CC_train = x_train['CC']
CC_train = CC_train.astype(np.uint8)

#### validation labels ###
Y_val = x_val['Y']
Y_val = Y_val.astype(np.uint8)
D_val = x_val['D']
D_val = D_val.astype(np.uint8)
CC_val = x_val['CC']
CC_val = CC_val.astype(np.uint8)

#### test labels ###
Y_test = x_test['Y']
Y_test = Y_test.astype(np.uint8)
D_test = x_test['D']
D_test = D_test.astype(np.uint8)
CC_test = x_test['CC']
CC_test = CC_test.astype(np.uint8)


#### Creating image values

In [9]:
# train
values = np.zeros(shape=(len(x_train),56,106,1))
i = 0
j = 0
while i < len(x_train):
    values[i] = x_train['img_value'][j]
    i += 1
    j+=1
x_train = values/255

# val
values = np.zeros(shape=(len(x_val),56,106,1))
i = 0
j = 7680
while i < len(x_val):
    values[i] = x_val['img_value'][j]
    i += 1
    j+=1
x_val = values/255

# test
values = np.zeros(shape=(len(x_test),56,106,1))
i = 0
j = 9600
while i < len(x_test):
    values[i] = x_test['img_value'][j]
    i += 1
    j+=1
x_test = values/255


### IMPORT 30K Data

In [19]:
Hee_csv = pd.read_csv('30_samples_labeled_csv.csv',header=None, names=["index", "string"],delimiter=';')
Hee_csv['CC'] = 0
Hee_csv['D'] = 0
Hee_csv['Y'] = 0
Hee_csv = Hee_csv.astype(str)
# Iterate string digits and append classes
for i, row in Hee_csv.iterrows():
    if len(row['string']) != 4:
        row['CC'] = '1'
        row['D'] = '10'
        row['Y'] = '10'
    else:
        row['D'] = row['string'][2]
        row['Y'] = row['string'][3]
        if row['string'][0:2] == '18':
            row['CC']='0'
        else:
            row['CC']='1'
#os.chdir(path_images)
image_array, filename = load_image_function('30_samples')

# and convert to a img_df
img_df = pd.DataFrame({'filename': filename, 'img_value': list(image_array)}, 
                        columns=['filename', 'img_value'])
# Create proper index value in img_df to allow merge on Hee_csv
img_df['index'] = img_df['filename']
for i, row in img_df.iterrows():
    row['index'] = str(img_df['index'][i]).split('.')[0]
img_df
# End of creating img_df
#
# Start: merge img_df with Hee_csv
# match index type on dataframes to merge
Hee_csv['index'] = Hee_csv['index'].astype(int)
img_df['index'] = img_df['index'].astype(int)
# Merge dataframes
df_img_classes = Hee_csv.merge(img_df)
# Rearrange order of dataframe
df_img_classes = df_img_classes.reindex(columns= ['index', 'string', 'CC', 'D', 'Y', 'img_value', 'filename'])
# End og merging data frames

In [20]:
k30_labels_CC = df_img_classes['CC']
k30_labels_D = df_img_classes['D']
k30_labels_Y = df_img_classes['Y']

In [21]:
k30_img_vals =  df_img_classes['img_value']
values = np.zeros(shape=(len(k30_img_vals),56,106,1))
i = 0
j = 0
while i < len(k30_img_vals):
    values[i] = k30_img_vals[j]
    i += 1
    j+=1
k30_img_vals = values/255

In [22]:
n_samples = len(k30_img_vals)
k30_img_vals_non_deep = k30_img_vals.reshape((n_samples, -1))
k30_img_vals_non_deep.shape

(1000, 5936)

## Question 1

### 1.2 Non-deeplearning CC-D-Y models

In [10]:
x_train_flat = x_train.reshape((len(x_train), -1))
x_val_flat = x_val.reshape((len(x_val), -1))
x_test_flat = x_test.reshape((len(x_test), -1))

#### SVM, modelCC

In [15]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
Cs = [1,2,3]
Ds = [1,2,3,4]
gammas = ['scale','auto']
decisions = ['ovr', 'ovo']
results = []

for kernel in kernels:
        if kernel == 'poly':
            for C in Cs:
                for D in Ds:
                    for gamma in gammas:
                        for decision in decisions:
                            svm_current = svm.SVC(kernel=kernel, 
                                                    degree=D,
                                                    C =C,
                                                    gamma = gamma,
                                                    random_state = 2022,
                                                    decision_function_shape = decision)
                            svm_current.fit(x_train_flat, CC_train)
                            val_hat = svm_current.predict(x_val_flat)
                            accuracy = accuracy_score(val_hat, CC_val)
                            #
                            results.append([accuracy, kernel, C, gamma, decision, D])
                            print([accuracy, kernel, C, gamma, decision, None])
        else:
            for C in Cs:
                for gamma in gammas:
                    for decision in decisions:
                        svm_current = svm.SVC(kernel=kernel,
                                              C=C,
                                              gamma = gamma,
                                              decision_function_shape = decision,
                                              random_state = 2022)
                        svm_current.fit(x_train_flat, CC_train)
                        val_hat = svm_current.predict(x_val_flat)
                        accuracy = accuracy_score(val_hat, CC_val)
                        #
                        results.append([accuracy, kernel, C, gamma, decision, None])
                        print([accuracy, kernel, C, gamma, decision, None])

results = pd.DataFrame(results)
results.columns = ['Accuracy', 'Kernel','C', 'Gamma', 'Decision', 'Degree']
results

#model = svm.SVC()
#model.fit(x_train_flat, CC_train)
#val_hat = model.predict(x_val_flat)
#accuracy = accuracy_score(val_hat, CC_val)*100
#accuracy

[0.9614583333333333, 'linear', 1, 'scale', 'ovr', None]
[0.9614583333333333, 'linear', 1, 'scale', 'ovo', None]
[0.9614583333333333, 'linear', 1, 'auto', 'ovr', None]
[0.9614583333333333, 'linear', 1, 'auto', 'ovo', None]
[0.9609375, 'linear', 2, 'scale', 'ovr', None]
[0.9609375, 'linear', 2, 'scale', 'ovo', None]
[0.9609375, 'linear', 2, 'auto', 'ovr', None]
[0.9609375, 'linear', 2, 'auto', 'ovo', None]
[0.9609375, 'linear', 3, 'scale', 'ovr', None]
[0.9609375, 'linear', 3, 'scale', 'ovo', None]
[0.9609375, 'linear', 3, 'auto', 'ovr', None]
[0.9609375, 'linear', 3, 'auto', 'ovo', None]
[0.9651041666666667, 'poly', 1, 'scale', 'ovr', None]
[0.9651041666666667, 'poly', 1, 'scale', 'ovo', None]
[0.9625, 'poly', 1, 'auto', 'ovr', None]
[0.9625, 'poly', 1, 'auto', 'ovo', None]
[0.9661458333333334, 'poly', 1, 'scale', 'ovr', None]
[0.9661458333333334, 'poly', 1, 'scale', 'ovo', None]
[0.9625, 'poly', 1, 'auto', 'ovr', None]
[0.9625, 'poly', 1, 'auto', 'ovo', None]
[0.9609375, 'poly', 1, 'sc

,Accuracy,Kernel,C,Gamma,Decision,Degree
0,0.961458,linear,1,scale,ovr,NaN
1,0.961458,linear,1,scale,ovo,NaN
2,0.961458,linear,1,auto,ovr,NaN
3,0.961458,linear,1,auto,ovo,NaN
4,0.960938,linear,2,scale,ovr,NaN
...,...,...,...,...,...,...
79,0.962500,sigmoid,2,auto,ovo,NaN
80,0.962500,sigmoid,3,scale,ovr,NaN
81,0.962500,sigmoid,3,scale,ovo,NaN
82,0.962500,sigmoid,3,auto,ovr,NaN


In [17]:
accuracy_done = accuracy_score(val_hat, CC_val)*100
accuracy_done

96.25

In [19]:
print(str(results[results['Accuracy'] == results['Accuracy'].max()]))

    Accuracy Kernel  C  Gamma Decision  Degree
68  0.972917    rbf  3  scale      ovr     NaN
69  0.972917    rbf  3  scale      ovo     NaN


In [34]:
svm_final = svm.SVC(kernel='rbf',
                      C=3,
                      gamma = 'scale',
                      decision_function_shape = 'ovr',
                      random_state = 2022)
svm_final.fit(np.concatenate((x_train_flat, x_val_flat), axis=0), np.concatenate((CC_train, CC_val), axis=0))
val_hat = svm_final.predict(x_val_flat)
accuracy = accuracy_score(val_hat, CC_val)
accuracy

0.9973958333333334

#### Catboost, modelD

In [25]:
results_catboost_D = []
n_iterations = [500]#,200,500]
lr = [0.1]#,0.01]
n_depth = [6]#,4]
loss_f = 'MultiClass'

for iters in n_iterations:
    for ls in lr:
        for depths in n_depth:
            model_D = CatBoostClassifier(iterations=iters,
                                        learning_rate=ls,
                                        depth=depths,
                                        loss_function=loss_f,
                                        task_type="GPU",
                                        devices='0:1')

            model_D.fit(x_train_flat, D_train, verbose=50)
            acc = model_D.score(x_val_flat,D_val)
            # the results from the different hyperparameters are then stored so we can get the best one
            results_catboost_D.append([acc,ls,iters, depths])
results_df_catboost_D = pd.DataFrame(results_catboost_D)
results_df_catboost_D.columns=['accuracy',"learning_rate","n_estimators", "depth"]
#prints the results from best to worst in regards to accuracy, listting the hyperparameters for the result
results_df_catboost_D = results_df_catboost_D.sort_values('accuracy', ascending=False)
results_df_catboost_D

0:	learn: 2.1675784	total: 320ms	remaining: 2m 39s
50:	learn: 1.2035216	total: 13.3s	remaining: 1m 56s
100:	learn: 1.0174397	total: 25.9s	remaining: 1m 42s
150:	learn: 0.8764784	total: 38.3s	remaining: 1m 28s
200:	learn: 0.7694848	total: 50.7s	remaining: 1m 15s
250:	learn: 0.6861388	total: 1m 2s	remaining: 1m 2s
300:	learn: 0.6157047	total: 1m 14s	remaining: 49.5s
350:	learn: 0.5527428	total: 1m 27s	remaining: 36.9s
400:	learn: 0.5027709	total: 1m 38s	remaining: 24.4s
450:	learn: 0.4594386	total: 1m 50s	remaining: 12s
499:	learn: 0.4204521	total: 2m 2s	remaining: 0us


,accuracy,learning_rate,n_estimators,depth
0,0.642188,0.1,500,6


In [18]:
results_df_catboost_D.to_csv('Catboost_results_DECADE')

#### Catboost, modelY

In [26]:
results = []
n_iterations = [500]#,200,500]
lr = [0.1]#,0.01]
n_depth = [6]#,4]
loss_f = 'MultiClass'

for iters in n_iterations:
    for ls in lr:
        for depths in n_depth:
            model_Y = CatBoostClassifier(iterations=iters,
                                        learning_rate=ls,
                                        depth=depths,
                                        loss_function=loss_f,
                                        task_type="GPU",
                                        devices='0:1')

            model_Y.fit(x_train_flat, Y_train, verbose=50)
            acc = model_Y.score(x_val_flat,Y_val)
            # the results from the different hyperparameters are then stored so we can get the best one
            results.append([acc,ls,iters, depths])
results_df = pd.DataFrame(results)
results_df.columns=['accuracy',"learning_rate","n_estimators", "depth"]
#prints the results from best to worst in regards to accuracy, listting the hyperparameters for the result
results_df = results_df.sort_values('accuracy', ascending=False)
results_df

0:	learn: 2.3615583	total: 321ms	remaining: 2m 40s
50:	learn: 1.7184887	total: 15.1s	remaining: 2m 13s
100:	learn: 1.4316729	total: 29.6s	remaining: 1m 56s
150:	learn: 1.2419622	total: 43.5s	remaining: 1m 40s
200:	learn: 1.1052926	total: 56.6s	remaining: 1m 24s
250:	learn: 0.9958752	total: 1m 9s	remaining: 1m 9s
300:	learn: 0.9056422	total: 1m 22s	remaining: 54.5s
350:	learn: 0.8293127	total: 1m 34s	remaining: 40.3s
400:	learn: 0.7668663	total: 1m 47s	remaining: 26.5s
450:	learn: 0.7151730	total: 1m 59s	remaining: 13s
499:	learn: 0.6703028	total: 2m 11s	remaining: 0us


,accuracy,learning_rate,n_estimators,depth
0,0.566667,0.1,500,6


In [19]:
results_df.to_csv('Catboost_results_YEAR')

In [27]:
def score_calc(model,x_test,y_test):
    score = 0.0
    preds = model.predict(x_test)
    i = 0
    for row in y_test:
        if str(row) == str(preds[i][0]):
            score = score + (1/3)
        i= i + 1
    return score

In [28]:
model_Y_score = score_calc(model_Y,x_test_flat,Y_test)
model_D_score = score_calc(model_D,x_test_flat,D_test)

In [29]:
print(model_Y_score)
print(model_D_score)

442.66666666665895
513.9999999999885


In [33]:
model_Y_score = score_calc(model_Y,k30_img_vals_non_deep,k30_labels_Y)
model_D_score = score_calc(model_D,k30_img_vals_non_deep,k30_labels_D)

In [34]:
print(model_Y_score)
print(model_D_score)

121.99999999999935
143.99999999999972


In [32]:
def sequence_calc(model_CC,model_D,model_Y,CC_test,D_test,Y_test,x_test):
    preds_D = model_D.predict(x_test)
    preds_Y = model_Y.predict(x_test)
    preds_CC = model_CC.predict(x_test)
    CC_array = np.array(CC_test)
    D_array = np.array(D_test)
    Y_array = np.array(Y_test)
    score = 0.0
    i = 0
    while i < len(CC_array):
        if int(CC_array[i]) == preds_CC[i]:
            if int(D_array[i]) == preds_D[i]:
                if int(Y_array[i]) == preds_Y[i]:
                    score = score + 1
        i= i + 1
    return score

In [ ]:
seq_score = sequence_calc(svm_final,model_D,model_Y,CC_test,D_test,Y_test,x_test_flat)
seq_score

895.0

In [ ]:
seq_score = sequence_calc(svm_final,model_D,model_Y,k30_labels_CC,k30_labels_D,k30_labels_Y,k30_img_vals_non_deep)
seq_score

160.0

## Question 2

In [2]:
#Normalization of gray values
gray_val = gray_val/255
gray_train = gray_train/255
gray_test = gray_test/255
rgb_val = rgb_val/255
rgb_train = rgb_train/255
rgb_test = rgb_test/255

NameError: name 'gray_val' is not defined

### 2.2.A Construct a CNN

### 2.2.B Regularization

### 2.2.C Augmentation

### 2.2.D Transfer learning

## 2.3 Prefered CNN model

## Question 3

### 3.1 Visualization of activation maps

### 3.2 Investigating model performance